In [ ]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/2024 K-NET 부없프/AI_Human.csv"

In [ ]:
import datasets

dataset = datasets.Dataset.from_csv(path).shuffle(seed=42)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['text', 'generated'],
    num_rows: 487235
})

In [ ]:
temp = dataset.train_test_split(test_size=0.4, seed=42)

In [ ]:
temp

DatasetDict({
    train: Dataset({
        features: ['text', 'generated'],
        num_rows: 292341
    })
    test: Dataset({
        features: ['text', 'generated'],
        num_rows: 194894
    })
})

In [ ]:
temp2 = temp['test'].train_test_split(test_size=0.5, seed=42)
temp2

DatasetDict({
    train: Dataset({
        features: ['text', 'generated'],
        num_rows: 97447
    })
    test: Dataset({
        features: ['text', 'generated'],
        num_rows: 97447
    })
})

In [ ]:
dataset = datasets.DatasetDict({
    "train": temp["train"],
    "valid": temp2["train"],
    "test": temp2["test"],
    })

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'generated'],
        num_rows: 292341
    })
    valid: Dataset({
        features: ['text', 'generated'],
        num_rows: 97447
    })
    test: Dataset({
        features: ['text', 'generated'],
        num_rows: 97447
    })
})

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
model = DistilBertModel.from_pretrained('distilbert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples): # BERT는 입력 토큰이 512개짜리인 것만 받을 수 있음
    return tokenizer(examples["text"], padding=True, truncation=True) # 모자라면 패딩, 넘치면 잘라내기

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True, num_proc=64)

Map (num_proc=64):   0%|          | 0/292341 [00:00<?, ? examples/s]

Map (num_proc=64):   0%|          | 0/97447 [00:00<?, ? examples/s]

Map (num_proc=64):   0%|          | 0/97447 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tokenized_datasets.push_to_hub("knet-gpt-detection-project", private=True)

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/98 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/98 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/98 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/98 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/98 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/615 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/cacbon-dioxit/knet-gpt-detection-project/commit/7bdb92abee4c070af6670c34cb27a2516705558a', commit_message='Upload dataset', commit_description='', oid='7bdb92abee4c070af6670c34cb27a2516705558a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/cacbon-dioxit/knet-gpt-detection-project', endpoint='https://huggingface.co', repo_type='dataset', repo_id='cacbon-dioxit/knet-gpt-detection-project'), pr_revision=None, pr_num=None)